---
title: Represent quantum computers
description: Learn about coupling maps, basis gates, and processor errors for transpiling
---


# Representing quantum computers for the transpiler

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>

To convert an abstract circuit to an ISA circuit that can run on a specific QPU (quantum processing unit), the transpiler needs certain information about the QPU. This information is found in two places: the `BackendV2` (or legacy `BackendV1`) object you plan to submit jobs to, and the backend's `Target` attribute.

- The [`Target`](../api/qiskit/qiskit.transpiler.Target) contains all the relevant constraints of a device, such as its native basis gates, qubit connectivity, and pulse or timing information.
- The [`Backend`](../api/qiskit/qiskit.providers.BackendV2) possesses a `Target` by default, contains additional information -- such as the [`InstructionScheduleMap`](/docs/api/qiskit/1.4/qiskit.pulse.InstructionScheduleMap), and provides the interface for submitting quantum circuit jobs.

You can also explicitly provide information for the transpiler to use, for example, if you have a specific use case, or if you believe this information will help the transpiler generate a more optimized circuit.


The precision with which the transpiler produces the most appropriate circuit for specific hardware depends on how much information the `Target` or `Backend` has about its constraints.

<Admonition type="note">
Because many of the underlying transpilation algorithms are stochastic, there is no guarantee that a better circuit will be found.
</Admonition>

This page shows several examples of passing QPU information to the transpiler. These examples use the target from the [`FakeSherbrooke`](/docs/api/qiskit-ibm-runtime/fake-provider-fake-sherbrooke#fakesherbrooke) mock backend.

<span id="default-config"></span>
## Default configuration

The simplest use of the transpiler is to provide all the QPU information by providing the `Backend` or `Target`. To better understand how the transpiler works, construct a circuit and transpile it with different information, as follows.

Import the necessary libraries and instantiate the QPU:
In order to convert an abstract circuit to an ISA circuit that can run on a specific processor, the transpiler needs certain information about the processor.  Typically, this information is stored in the [`Backend`](/docs/api/qiskit/qiskit.providers.Backend#backend) or [`Target`](/docs/api/qiskit/qiskit.transpiler.Target#target) provided to the transpiler, and no further information is needed. However, you can also explicitly provide information for the transpiler to use, for example, if you have a specific use case, or if you believe this information will help the transpiler generate a more optimized circuit.

This topic shows several examples of passing information to the transpiler. These examples use the target from the [`FakeSherbrooke`](/docs/api/qiskit-ibm-runtime/fake-provider-fake-sherbrooke#fakesherbrooke) mock backend.

In [1]:
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

backend = FakeSherbrooke()
target = backend.target

The example circuit uses an instance of [`efficient_su2`](/docs/api/qiskit/qiskit.circuit.library.efficient_su2) from Qiskit's circuit library.

In [2]:
from qiskit.circuit.library import efficient_su2

qc = efficient_su2(12, entanglement="circular", reps=1)

qc.draw("mpl")

<Image src="/docs/images/guides/represent-quantum-computers/extracted-outputs/97f9acc1-ac53-4025-b413-485777932a9b-0.svg" alt="Output of the previous code cell" />

This example uses default settings to transpile to the `backend`'s `target`, which provides all the information needed to convert the circuit to one that will run on the backend.

In [3]:
from qiskit.transpiler import generate_preset_pass_manager

pass_manager = generate_preset_pass_manager(
    optimization_level=1, target=target, seed_transpiler=12345
)
qc_t_target = pass_manager.run(qc)
qc_t_target.draw("mpl", idle_wires=False, fold=-1)

<Image src="/docs/images/guides/represent-quantum-computers/extracted-outputs/4b81fb9d-d199-45c5-b119-c1f0b973afe9-0.svg" alt="Output of the previous code cell" />

This example is used in later sections of this topic to illustrate that the coupling map and basis gates are the essential pieces of information to pass to the transpiler for optimal circuit construction. The QPU can usually select default settings for other information that is not passed in, such as timing and scheduling.

## Coupling map

The coupling map is a graph that shows which qubits are connected and hence have two-qubit gates between them. Sometimes this graph is directional, meaning that the two-qubit gates can only go in one direction. However, the transpiler can always flip a gate's direction by adding additional single-qubit gates. An abstract quantum circuit can always be represented on this graph, even if its connectivity is limited, by introducing SWAP gates to move the quantum information around.

The qubits from our abstract circuits are called _virtual qubits_ and those on the coupling map are _physical qubits_. The transpiler provides a mapping between virtual and physical qubits. One of the first steps in transpilation, the _layout_ stage, performs this mapping.

<Admonition type="note">
Although the routing stage is intertwined with the _layout_ stage — which selects the actual qubits — by default, this topic treats them as separate stages for simplicity. The combination of routing and layout is called _qubit mapping_.  Learn more about these stages in the [Transpiler stages](transpiler-stages) topic.
</Admonition>

Pass the `coupling_map` keyword argument to see its effect on the transpiler:

In [4]:
coupling_map = target.build_coupling_map()

pass_manager = generate_preset_pass_manager(
    optimization_level=0, coupling_map=coupling_map, seed_transpiler=12345
)
qc_t_cm_lv0 = pass_manager.run(qc)
qc_t_cm_lv0.draw("mpl", idle_wires=False, fold=-1)

<Image src="/docs/images/guides/represent-quantum-computers/extracted-outputs/ec354bee-e06b-42ea-a117-6c1a9308ca73-0.svg" alt="Output of the previous code cell" />

As shown above, several SWAP gates were inserted (each consisting of three CX gates), which will cause a lot of errors on current devices. To see which qubits are selected on the actual qubit topology, use `plot_circuit_layout` from Qiskit Visualizations:

In [5]:
from qiskit.visualization import plot_circuit_layout

plot_circuit_layout(qc_t_cm_lv0, backend, view="physical")

<Image src="/docs/images/guides/represent-quantum-computers/extracted-outputs/9be74535-ed36-4d51-afeb-ee53c3f8a046-0.svg" alt="Output of the previous code cell" />

This shows that our virtual qubits 0-11 were trivially mapped to the line of physical qubits 0-11. Let's return to the default (`optimization_level=1`), which uses `VF2Layout` if any routing is required.

In [6]:
pass_manager = generate_preset_pass_manager(
    optimization_level=1, coupling_map=coupling_map, seed_transpiler=12345
)
qc_t_cm_lv1 = pass_manager.run(qc)
qc_t_cm_lv1.draw("mpl", idle_wires=False, fold=-1)

<Image src="/docs/images/guides/represent-quantum-computers/extracted-outputs/8035fd05-f7cd-4151-b19a-4968202246e6-0.svg" alt="Output of the previous code cell" />

Now there are no SWAP gates inserted and the physical qubits selected are the same when using the `target` class.

In [7]:
from qiskit.visualization import plot_circuit_layout

plot_circuit_layout(qc_t_cm_lv1, backend, view="physical")

<Image src="/docs/images/guides/represent-quantum-computers/extracted-outputs/25d9fac3-abda-4b2d-81b4-351dc0772722-0.svg" alt="Output of the previous code cell" />

Now the layout is in a ring.  Because this layout respects the circuit's connectivity, there are no SWAP gates, providing a much better circuit for execution.

## Basis gates

Every quantum computer supports a limited instruction set, called its _basis gates_.  Every gate in the circuit must be translated to the elements of this set. This set should consist of single- and two-qubit gates that provide a universal gates set, meaning that any quantum operation can be decomposed into those gates.  This is done by the [BasisTranslator](../api/qiskit/qiskit.transpiler.passes.BasisTranslator), and the basis gates can be specified as a keyword argument to the transpiler to provide this information.

In [8]:
basis_gates = list(target.operation_names)
print(basis_gates)

['sx', 'switch_case', 'x', 'if_else', 'measure', 'for_loop', 'delay', 'ecr', 'id', 'reset', 'rz']


The default single-qubit gates on _ibm_sherbrooke_ are `rz`, `x`, and `sx`, and the default two-qubit gate is `ecr` (echoed cross-resonance). CX gates are constructed from `ecr` gates, so on some QPUs `ecr` is specified as the two-qubit basis gate, while on others `cx` is the default. The `ecr` gate is the _entangling_ part of the `cx` gate. In addition to the control gates, there are also `delay` and `measurement` instructions.


<Admonition type="note">
    QPUs have default basis gates, but you can choose whatever gates you want, as long as you provide the instruction or add pulse gates (see [Create transpiler passes.](custom-transpiler-pass)) The default basis gates are those that calibrations have been done for on the QPU, so no further instruction/pulse gates need to be provided. For example, on some QPUs `cx` is the default two-qubit gate and `ecr` on others. See the list of possible [native gates and operations](/docs/guides/qpu-information#native-gates) for more details.
</Admonition>

In [9]:
pass_manager = generate_preset_pass_manager(
    optimization_level=1,
    coupling_map=coupling_map,
    basis_gates=basis_gates,
    seed_transpiler=12345,
)
qc_t_cm_bg = pass_manager.run(qc)
qc_t_cm_bg.draw("mpl", idle_wires=False, fold=-1)

<Image src="/docs/images/guides/represent-quantum-computers/extracted-outputs/313e4743-0.svg" alt="Output of the previous code cell" />

Note that the `CXGate` objects have been decomposed to `ecr` gates and single-qubit basis gates.

## Device error rates

The `Target` class can contain information about the error rates for operations on the device.
For example, the following code retrieves the properties for the echoed cross-resonance (ECR) gate between qubit 1 and 0 (note that the ECR gate is directional):

In [10]:
target["ecr"][(1, 0)]

InstructionProperties(duration=5.333333333333332e-07, error=0.007494257741828603)

The output displays the duration of the gate (in seconds) and its error rate. To reveal error information to the transpiler, build a target model with the `basis_gates` and `coupling_map` from above and populate it with error values from the backend `FakeSherbrooke`.

In [11]:
from qiskit.transpiler import Target
from qiskit.circuit.controlflow import IfElseOp, SwitchCaseOp, ForLoopOp

err_targ = Target.from_configuration(
    basis_gates=basis_gates,
    coupling_map=coupling_map,
    num_qubits=target.num_qubits,
    custom_name_mapping={
        "if_else": IfElseOp,
        "switch_case": SwitchCaseOp,
        "for_loop": ForLoopOp,
    },
)

for i, (op, qargs) in enumerate(target.instructions):
    if op.name in basis_gates:
        err_targ[op.name][qargs] = target.instruction_properties(i)

Transpile with our new target `err_targ` as the target:

In [12]:
pass_manager = generate_preset_pass_manager(
    optimization_level=1, target=err_targ, seed_transpiler=12345
)
qc_t_cm_bg_et = pass_manager.run(qc)
qc_t_cm_bg_et.draw("mpl", idle_wires=False, fold=-1)

<Image src="/docs/images/guides/represent-quantum-computers/extracted-outputs/f1e270c4-e2cc-487e-a050-4180bc321b0b-0.svg" alt="Output of the previous code cell" />

Because the target includes error information, the `VF2PostLayout` pass tries to find the optimal qubits to use, resulting in the same circuit that was originally found with the same physical qubits.

## Next steps

<Admonition type="tip" title="Recommendations">
    - Understand [Transpilation default settings and configuration options.](defaults-and-configuration-options)
    - Review the [Commonly used parameters for transpilation](common-parameters) topic.
    - Try the [Compare transpiler settings](/docs/guides/circuit-transpilation-settings#compare-transpiler-settings) guide.
    - See the [Transpile API documentation.](/docs/api/qiskit/transpiler)
</Admonition>